In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("../../data/18_channels_2023_09_to_2024_09.parquet")
print(df.shape)
df.start = pd.to_datetime(df.start)
cutoff = df.start.max() - pd.Timedelta(days=21)
df = df[df.start >= cutoff]
print(df.shape)

(185738, 10)
(3022, 10)


In [3]:
display(df.text.str.len().describe())
display(df.num_tokens.describe())

display(df.channel_is_radio.value_counts())
display(df.channel_program_type.value_counts())

count    3022.000000
mean     2133.906023
std       386.480849
min       344.000000
25%      1972.250000
50%      2196.500000
75%      2382.000000
max      3144.000000
Name: text, dtype: float64

count    3022.000000
mean      475.428855
std        85.835609
min       106.000000
25%       437.000000
50%       488.000000
75%       530.000000
max       711.000000
Name: num_tokens, dtype: float64

channel_is_radio
False    1847
True     1175
Name: count, dtype: int64

channel_program_type
Information en continu            1811
Information - Magazine             748
Information - Journal              397
Information - Autres émissions      66
Name: count, dtype: int64

In [4]:
import litellm
from litellm import acompletion, completion_cost
from pydantic import BaseModel, Field
from typing import Callable, Awaitable, Union
from functools import wraps
import asyncio
from tqdm.asyncio import tqdm


class MediatreePrediction(BaseModel):
    cards_label_pred: str
    prompt_tokens: int
    completion_tokens: int
    total_tokens: int
    cost: float
    claim_pred: str | None = None


class MediatreeClaimIdentifier(BaseModel):
    claim: str = Field(
        description="Reformulation courte et claire de l'affirmation à vérifier"
    )
    context: str = Field(
        description="Un seul paragraphe résumant le contexte essentiel pour comprendre l'affirmation"
    )
    quote: str = Field(description="Citation exacte du texte contenant l'affirmation")


class MediatreeClaimIdentifierResponse(BaseModel):
    claims: Union[list[MediatreeClaimIdentifier], None]
    prompt_tokens: Union[int, None]
    completion_tokens: Union[int, None]
    total_tokens: Union[int, None]
    cost: Union[float, None]


async def report_experiment_results(
    df: pd.DataFrame, predict_experiment: Callable[[str], Awaitable[None]]
) -> None:
    # Copy df to avoid modifications
    df = df.copy()

    # Run the experiment
    mediatree_predictions: list[MediatreeClaimIdentifierResponse] = await tqdm.gather(
        *[predict_experiment(text) for text in df["text"]]
    )
    # Create lists to store individual claims and their metadata
    rows = []

    for idx, pred in zip(df.index, mediatree_predictions):
        if pred is not None and pred.claims is not None:
            for claim in pred.claims:
                rows.append(
                    {
                        "id": idx,
                        "claim": claim.claim,
                        "context": claim.context,
                        "quote": claim.quote,
                        "prompt_tokens": pred.prompt_tokens,
                        "completion_tokens": pred.completion_tokens,
                        "total_tokens": pred.total_tokens,
                        "cost": pred.cost,
                    }
                )

    # Create DataFrame with one claim per row
    mediatree_predictions_df = pd.DataFrame(rows)

    if not mediatree_predictions_df.empty:
        # Set id as index if there are any claims
        mediatree_predictions_df = mediatree_predictions_df.set_index("id")
    # df = pd.concat([df, mediatree_predictions_df], axis=1)
    df = df.merge(
        mediatree_predictions_df, left_index=True, right_index=True, how="outer"
    )

    # Show stats and performance
    show_llm_usage(df)

    return df


def show_llm_usage(df: pd.DataFrame) -> None:
    print("\nLLM USAGE\n=========\n")
    print(
        f"Median token usage:\n- Prompt: {int(df["prompt_tokens"].median())}\n"
        f"- Completion: {int(df["completion_tokens"].median())}\n"
        f"- Total: {int(df["total_tokens"].median())}"
    )
    print(f"\nTotal cost: ${df["cost"].sum():.3f}\n")


# Limit concurrent requests to avoid API rate limiting
# (it depends on the model you use and your API tier)
semaphore = asyncio.Semaphore(5)


# Decorator that ensures `acompletion` uses the semaphore
def with_semaphore(acquire_semaphore):
    def decorator(func):
        @wraps(func)
        async def wrapper(*args, **kwargs):
            async with acquire_semaphore:
                return await func(*args, **kwargs)

        return wrapper

    return decorator


acompletion = with_semaphore(semaphore)(acompletion)

In [5]:
import json


async def say(text, sec):
    await asyncio.sleep(sec)
    print(text)


async def extract_claims(text: str) -> MediatreeClaimIdentifierResponse:
    system_prompt = """
Tu es un assistant spécialisé dans l'analyse de désinformation environnementale.

TÂCHE:
Analyse l'extrait de transcription TV/Radio fourni et identifie les affirmations (claims) qui nécessitent une vérification factuelle sur les thèmes suivants:
- Changement climatique
- Transition écologique
- Énergie
- Biodiversité
- Pollution
- Pesticides
- Ressources naturelles (eau, minéraux, etc.)

FORMAT DE RÉPONSE:
Tu dois OBLIGATOIREMENT répondre au format JSON suivant:
{
    "claims": [
        {
            "claim": "Reformulation courte et claire de l'affirmation à vérifier",
            "context": "Un seul paragraphe résumant le contexte essentiel pour comprendre l'affirmation",
            "quote": "Citation exacte du texte contenant l'affirmation"
        }
    ]
}

RÈGLES IMPORTANTES:
1. Inclure UNIQUEMENT les affirmations vérifiables sur les thèmes environnementaux
2. Chaque claim doit être unique
3. Le format JSON doit être strictement respecté
4. Si aucune affirmation à vérifier n'est trouvée, renvoyer un tableau claims vide
5. Maximum 3 claims par analyse

Analyse maintenant le texte suivant:"""
    response = await acompletion(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text.strip()},
        ],
        model="gpt-4o-mini",
        max_tokens=2000,
        temperature=0,
    )
    claim_identifier = response.choices[0].message.content
    claims = (
        [
            MediatreeClaimIdentifier(**claim)
            for claim in json.loads(claim_identifier)["claims"]
        ]
        if json.loads(claim_identifier)["claims"] != []
        else None
    )
    return (
        MediatreeClaimIdentifierResponse(
            claims=claims,
            prompt_tokens=response.usage.prompt_tokens,
            completion_tokens=response.usage.completion_tokens,
            total_tokens=response.usage.total_tokens,
            cost=completion_cost(response),
        )
        if claims is not None
        else None
    )


claim_detections = await report_experiment_results(df.iloc[:300], extract_claims)

100%|██████████| 300/300 [02:20<00:00,  2.13it/s]


LLM USAGE

Median token usage:
- Prompt: 754
- Completion: 296
- Total: 1052

Total cost: $0.087



In [6]:
claim_detections = claim_detections.dropna(subset=["claim", "context", "quote"])

In [7]:
claim_detections.shape

(308, 17)

In [8]:
claim_detections.to_csv("../../data/claim_detections_video_extractions.csv")

In [9]:
from llama_index.core import PromptTemplate

In [10]:
SYSTEM_PROMPT = """
QU'EST-CE QU'UNE AFFIRMATION (CLAIM)?
Une affirmation est une déclaration qui:
- Présente un fait ou une statistique vérifiable
- Fait une prédiction ou établit une relation de cause à effet
- Prend position sur un sujet environnemental (directement ou indirectement)
- Peut influencer l'opinion publique ou les décisions politiques

POURQUOI C'EST IMPORTANT?
La désinformation environnementale peut:
- Retarder des actions climatiques urgentes
- Influencer négativement les politiques publiques
- Créer de la confusion dans le débat public
- Avoir des conséquences réelles sur l'environnement

RÔLE:
Tu es un analyste spécialisé dans la vérification des affirmations environnementales, capable d'identifier les liens directs ET indirects avec l'environnement.

CRITÈRES DE VALIDATION:
Une affirmation est valide si elle concerne DIRECTEMENT ou INDIRECTEMENT:

Thèmes directs:
- Changement climatique
- Transition écologique
- Énergie
- Biodiversité
- Pollution
- Pesticides
- Ressources naturelles

Thèmes indirects (exemples):
- Empreinte carbone
- Catastrophes naturelles
- Énergies fossiles
- Transport et mobilité
- Agriculture et alimentation
- Météo extrême
- Urbanisme et aménagement
- Consommation et déchets
- Santé environnementale

FORMAT DE RÉPONSE OBLIGATOIRE:
Tu dois absolument répondre avec un objet JSON ayant cette structure:
{
    "valid": true/false,
    "explanation": "Ton explication en 1-2 phrases",
    "confidence_score": 0.0-1.0
}

RÈGLES IMPORTANTES:
- Le champ 'valid' doit être true ou false (sans guillemets)
- L'explication doit mentionner si le lien est direct ou indirect
- Le confidence_score doit être un nombre entre 0.0 et 1.0
- Respecte strictement le format JSON

TÂCHE:
Analyse l'affirmation suivante et détermine si elle est liée à l'environnement (directement ou indirectement).
"""

USER_PROMPT_TEMPLATE = """
ÉLÉMENTS À ANALYSER:
Affirmation: {claim}
Contexte: {context}
Citation exacte: {quote}
"""


class MediatreeClaimIdentifierValidation(BaseModel):
    valid: bool
    explanation: str
    confidence_score: float


# prompt_temp = PromptTemplate(user_prompt_template)
async def double_check_claim(
    system_prompt: str, user_prompt: str
) -> MediatreeClaimIdentifierValidation:
    response = await acompletion(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        model="gpt-4o-mini",
        max_tokens=500,
        temperature=0,
    )
    claim_validity = response.choices[0].message.content
    return MediatreeClaimIdentifierValidation(**json.loads(claim_validity))

In [11]:
async def validate_claims(claims_df: pd.DataFrame) -> pd.DataFrame:
    claim_validations: list[MediatreeClaimIdentifierValidation] = await tqdm.gather(
        *[
            double_check_claim(
                SYSTEM_PROMPT,
                USER_PROMPT_TEMPLATE.format(
                    claim=claim.claim, context=claim.context, quote=claim.quote
                ),
            )
            for claim in claims_df.itertuples()
        ]
    )

    claim_validations_df = pd.DataFrame(
        [validation.model_dump(exclude_none=True) for validation in claim_validations],
        index=claims_df.index,
    )

    claim_validations_df = pd.concat([claims_df, claim_validations_df], axis=1)

    return claim_validations_df

In [12]:
claim_validations_df = await validate_claims(claim_detections)

claim_validations_df.head()

100%|██████████| 308/308 [01:40<00:00,  3.07it/s]


,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,claim,context,quote,prompt_tokens,completion_tokens,total_tokens,cost,valid,explanation,confidence_score
id,,,,,,,,,,,,,,,,,,,,
0124841dd10fb23b834fe4a24a8b313da6e01e812b508eea29ca832a42b38497,2024-08-24 07:12:00,à l' heure la sécheresse on en parlait qui tou...,bfmtv,False,Information en continu,Information en continu,"[""biodiversite_solutions"", ""changement_climati...","[{""keyword"": ""restriction"", ""timestamp"": 17244...",1,531,Le niveau des nappes phréatiques est au plus b...,L'extrait décrit une situation de sécheresse à...,où le niveau des nappes phréatiques est au plu...,804.0,307.0,1111.0,0.000305,True,L'affirmation est liée à l'environnement de ma...,0.9
0124841dd10fb23b834fe4a24a8b313da6e01e812b508eea29ca832a42b38497,2024-08-24 07:12:00,à l' heure la sécheresse on en parlait qui tou...,bfmtv,False,Information en continu,Information en continu,"[""biodiversite_solutions"", ""changement_climati...","[{""keyword"": ""restriction"", ""timestamp"": 17244...",1,531,Les habitants de Durban-Corbières font face à ...,Les restrictions d'eau imposées aux habitants ...,les six cent soixante habitants de durban <unk...,804.0,307.0,1111.0,0.000305,True,L'affirmation est liée à la gestion des ressou...,0.8
0124841dd10fb23b834fe4a24a8b313da6e01e812b508eea29ca832a42b38497,2024-08-24 07:12:00,à l' heure la sécheresse on en parlait qui tou...,bfmtv,False,Information en continu,Information en continu,"[""biodiversite_solutions"", ""changement_climati...","[{""keyword"": ""restriction"", ""timestamp"": 17244...",1,531,Le coût d'approvisionnement en eau pour la com...,L'extrait mentionne le coût d'approvisionnemen...,chaque camion qui vient approvisionner le rése...,804.0,307.0,1111.0,0.000305,True,L'affirmation est liée à l'environnement de ma...,0.8
02d446693f5f911956586f4e8d2365c8b46f4f28317129d4ce9c641f7e65fc15,2024-08-23 13:50:00,lancer dans une improvisation je crois qu' il ...,bfmtv,False,Information en continu,Information en continu,"[""attenuation_climatique_solutions"", ""changeme...","[{""keyword"": ""\u00e9cologique"", ""timestamp"": 1...",3,488,Les vacances à vélo sont de plus en plus chois...,L'extrait mentionne que de plus en plus de per...,vous êtes de plus en plus nombreux à choisir c...,761.0,107.0,868.0,0.000178,True,L'affirmation établit un lien direct avec l'en...,0.9
0830e16bee072fb3e3422599141e217b5d39ecb9d05a84c7c970461c6cc50845,2024-08-29 06:10:00,pour retrouver et punir sévèrement les coupabl...,bfmtv,False,Information en continu,Information en continu,"[""changement_climatique_constat"", ""biodiversit...","[{""keyword"": ""cancer"", ""timestamp"": 1724904719...",1,554,La floraison de l'ambroisie intervient de plus...,L'affirmation fait référence à l'impact du réc...,l' allergie à l' ambroisie c' est une mauvaise...,827.0,312.0,1139.0,0.000311,True,L'affirmation établit un lien direct entre le ...,0.9


In [13]:
claim_validations_df.to_csv("../../data/claim_detections_validated.csv")

In [14]:
claim_validations_df.shape

(308, 20)

In [15]:
(
    claim_validations_df.loc[~claim_validations_df.valid].shape,
    claim_validations_df.loc[claim_validations_df.valid].shape,
)

((14, 20), (294, 20))

In [27]:
df_old_prompt = pd.read_parquet(
    "../../data/3_channels_predictions_09_2023_09_2024.parquet"
).iloc[:300]

In [40]:
rows = []
for idx, row in zip(df_old_prompt.index, df_old_prompt.to_dict(orient="records")):
    claims = row["claims"]
    for claim in claims:
        rows.append(
            {
                "id": idx,
                "claim": claim["claim"],
                "analysis": claim["analysis"],
                "context": claim["context"],
                "contradiction": claim["contradiction"],
                "disinformation_category": claim["disinformation_category"],
                "disinformation_score": claim["disinformation_score"],
                "pro_anti": claim["pro_anti"],
                "quote": claim["quote"],
                "speaker": claim["speaker"],
            }
        )

df_old_prompt_claims = pd.DataFrame(rows)

if not df_old_prompt_claims.empty:
    # Set id as index if there are any claims
    df_old_prompt_claims = df_old_prompt_claims.set_index("id")
# df = pd.concat([df, mediatree_predictions_df], axis=1)
df_old_prompt_claims = df_old_prompt.merge(
    df_old_prompt_claims, left_index=True, right_index=True, how="outer"
)

In [43]:
df_old_prompt_claims.columns

Index(['start', 'text', 'channel_name', 'channel_is_radio',
       'channel_program_type', 'channel_program', 'themes', 'keywords',
       'num_keywords', 'num_tokens', 'claims', 'claim', 'analysis', 'context',
       'contradiction', 'disinformation_category', 'disinformation_score',
       'pro_anti', 'quote', 'speaker'],
      dtype='object')

In [47]:
async def validate_claims_old_df(df: pd.DataFrame) -> pd.DataFrame:
    claim_validations: list[MediatreeClaimIdentifierValidation] = await tqdm.gather(
        *[
            double_check_claim(
                SYSTEM_PROMPT,
                USER_PROMPT_TEMPLATE.format(
                    claim=claim.analysis, context=claim.context, quote=claim.claim
                ),
            )
            for claim in df.itertuples()
        ]
    )

    claim_validations_df = pd.DataFrame(
        [validation.model_dump(exclude_none=True) for validation in claim_validations],
        index=df.index,
    )

    claim_validations_df = pd.concat([df, claim_validations_df], axis=1)

    return claim_validations_df

In [48]:
df_old_prompt_claims.text.iloc[0]

"il faut donc maintenant redéployer nos troupes elle est concentrée sur une seule position forte qui pourrait donc être le tchad sébastien le belgique et puis si la france souffre de la chaleur quarante et un des packs quarante quatre départements placés en vigilance canicule la grèce affronte des pluies torrentielles elles ont déjà fait un mort selon un bilan provisoire morgan parra londres pour une vente aux enchères qui fait déjà saliver les fans du groupe queen de leur défunt chanteur freddie mercury dimitri quelques notes de ce piano acquis quelques notes en de ce mille piano acquis neuf en mille neuf cents cent soixante quinze et sur lequel le chanteur a composé bohemian sera spread le piano c'est le piano qu'on entend le piano on entend absolument il est en vente chez sothebys avec bien d'autres objets des manuscrits plein de choses estimation entre deux et trois millions et demi d'euros enfin les sports fabien galthié dévoilera cet après midi la liste des joueurs retenus u pour

In [61]:
claim_validations_df = await validate_claims_old_df(df_old_prompt_claims.dropna(subset=["analysis", "context", "claim"]))

  0%|          | 0/440 [00:00<?, ?it/s]

100%|██████████| 440/440 [02:32<00:00,  2.88it/s]


In [62]:
claim_validations_df[
    [
        "claim",
        "analysis",
        "context",
        "valid",
        "explanation",
        "confidence_score",
    ]
].iloc[:100].to_csv("../../data/3_channels_predictions_09_2023_09_2024_validated_500.csv")

In [63]:
pd.options.display.max_columns = 999
claim_validations_df.head()

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,claims,claim,analysis,context,contradiction,disinformation_category,disinformation_score,pro_anti,quote,speaker,valid,explanation,confidence_score
id,,,,,,,,,,,,,,,,,,,,,,,
018b4d5809363be4c32dadb59ae447e4bd40fe49530ad910b8ea69bb4f402649,2023-09-06 07:08:00,il faut donc maintenant redéployer nos troupes...,europe1,True,Information - Magazine,Europe 1 Matin,"[""ressources_indirectes"", ""biodiversite_causes...","[{""keyword"": ""zinc"", ""timestamp"": 169397699804...",1,531,[{'analysis': 'Cette allégation est vérifiable...,La France souffre de la chaleur avec 41 départ...,Cette allégation est vérifiable et s'inscrit d...,Dans un segment sur les conditions climatiques...,N/A,facts,very low,N/A,La France souffre de la chaleur avec 41 départ...,N/A,True,L'affirmation est directement liée à l'environ...,0.90
018b4d5809363be4c32dadb59ae447e4bd40fe49530ad910b8ea69bb4f402649,2023-09-06 07:08:00,il faut donc maintenant redéployer nos troupes...,europe1,True,Information - Magazine,Europe 1 Matin,"[""ressources_indirectes"", ""biodiversite_causes...","[{""keyword"": ""zinc"", ""timestamp"": 169397699804...",1,531,[{'analysis': 'Cette allégation est vérifiable...,La Grèce affronte des pluies torrentielles qui...,"Similaire à l'accusation précédente, ceci a l'...",En parlant des événements climatiques en Europ...,N/A,facts,very low,N/A,La Grèce affronte des pluies torrentielles qui...,N/A,True,L'affirmation concerne directement un événemen...,0.90
03650f48569fffec2a3db5a6de6db9cf8238b26592447e1f6edfc6242d7b6558,2023-09-03 07:48:00,photos de votre chat finalement en version euh...,europe1,True,Information - Magazine,Europe 1 Matin Week-end,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169372014903...",1,568,[{'analysis': 'Cette déclaration semble étayer...,l'île de Mayotte vit en ce moment la pire séch...,Cette déclaration semble étayer une réalité en...,"Dans un bulletin d'information, il a été menti...",N/A,facts,very low,neutral,l'île vit en ce moment la pire sécheresse depu...,N/A,True,L'affirmation est liée à l'environnement de ma...,0.90
0713ddb459b76473c8f8f17ed92dc81d0869afd3b5c5f43099834f54476eb7d5,2023-09-06 06:46:00,mais aussi son public disons que euh on est da...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""ressources...","[{""keyword"": ""environnement"", ""timestamp"": 169...",2,499,[{'analysis': 'L'allégation sur l'empreinte ca...,la fabrication des consoles et des ordinateurs...,L'allégation sur l'empreinte carbone des conso...,Un intervenant discute de l'impact environneme...,N/A,facts,very low,pro-écologie,la fabrication des consoles et des ordinateurs...,N/A,True,L'affirmation est liée à l'environnement de ma...,0.90
0713ddb459b76473c8f8f17ed92dc81d0869afd3b5c5f43099834f54476eb7d5,2023-09-06 06:46:00,mais aussi son public disons que euh on est da...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""ressources...","[{""keyword"": ""environnement"", ""timestamp"": 169...",2,499,[{'analysis': 'L'allégation sur l'empreinte ca...,ces matières premières sont difficilement récu...,Ceci est une affirmation qui peut être vérifié...,Le même intervenant évoque la difficulté de ré...,N/A,facts,low,pro-écologie,ces matières premières sont difficilement récu...,N/A,True,L'affirmation établit un lien indirect avec l'...,0.85


valid
True     417
False     84
Name: count, dtype: int64